In [32]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
import json #https://stackoverflow.com/questions/7100125/storing-python-dictionaries
import pickle #https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
import os

In [5]:
files = os.listdir("models")
glove_models = []
vec_models = []
bin_models = []
for file in files:
    last_letter = file[-1]
    if last_letter == 't':
        glove_models += [file]
    elif last_letter == 'c':
        vec_models += [file]
    else:
        bin_models += [file]

In [6]:
vec_models, glove_models,bin_models

(['crawl-300d-2M-subword.vec',
  'crawl-300d-2M.vec',
  'wiki-news-300d-1M.vec',
  'wiki-news-300d-1M-subword.vec'],
 ['glove.6B.300d.txt',
  'glove.6B.200d.txt',
  'glove.42B.300d.txt',
  'glove.twitter.27B.50d.txt',
  'glove.6B.100d.txt',
  'glove.840B.300d.txt',
  'glove.6B.50d.txt',
  'glove.twitter.27B.200d.txt',
  'glove.twitter.27B.100d.txt',
  'glove.twitter.27B.25d.txt'],
 ['GoogleNews-vectors-negative300.bin'])

#### DEFINICAO DA ESTRUTURA DO NOSSO DICIONARIO
- Sentiment (somente retirados do nosso faamooooso array já salvo)
    - vectors
    - key_list

In [41]:
sentiments = np.load("./sentiments_list.npy")

In [37]:
def load_model(name, format_option): #(0,vec),(1,gloove),(2,bin)
    
    model_name = "./models/" + name
    if format_option == 0:
        model = KeyedVectors.load_word2vec_format(model_name)
    elif format_option == 1:
        tmp_file = get_tmpfile("tmp")
        _ = glove2word2vec(model_name, tmp_file)
        model = KeyedVectors.load_word2vec_format(tmp_file)
    else:
        model = KeyedVectors.load_word2vec_format(model_name, binary=True)
    print("Model " + name + " Loaded")jl
    return model

In [22]:
def generate_model_embeddings(model):
    model_embeddings = dict()
    for i,s in enumerate(sentiments):
        printPercentage(i,len(sentiments),10)
        top_similar = model.most_similar(positive=s,topn=50)
        sentiment_embeddings = dict()
        sentiment_embeddings["vectors"] = dict()
        sentiment_embeddings["key_list"] = top_similar
        sentiment_embeddings["vectors"][s] = model[s]
        for word,_ in top_similar:
            sentiment_embeddings["vectors"][word] = model[word]
        model_embeddings[s] = sentiment_embeddings
    if(len(model_embeddings.keys())!=len(sentiments)):
        print("erro no tamanho do embedding gerado")
    return model_embeddings

In [25]:
def write_embeddings_to_file(filename, model_embeddings):
    with open("./sentiments_embeddings/"+filename+'.pickle', 'wb') as handle:
        pickle.dump(model_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
model = load_model(bin_models[0],2)


Model GoogleNews-vectors-negative300.bin Loaded


In [ ]:
emb = generate_model_embeddings(model)
write_embeddings_to_file(glove_models[0],emb)

0.0%


0.0%
9.21%
18.42%
27.63%
36.84%
46.05%
55.26%
64.47%
73.68%
82.89%
92.11%
finished


### Help Methods And Cells

In [20]:
with open(model_name+'.pickle', 'rb') as handle:
    b = pickle.load(handle)

NameError: name 'model_name' is not defined

In [19]:
def printPercentage(i,size,steps):
    if i%(size//steps) == 0:
        print(str(round((i/size)*100,2)) + "%")
    if(i==size-1):
        print("finished")